# Data wrangling

In [43]:
import pandas as pd
import numpy as np

In [44]:
columns_name=['Date','Encode','Time','Price','Qty']
# read the single file
#/Users/yujiewang/MyProjects/Data Science/Data Science Mini-project/Project/Dataset/HSBC_Set01/Tapes
date_01_02=pd.read_csv('./Dataset/TstUoB_2024-01-02tapes.csv',names=columns_name,header=None)
date_01_02["State"]=" "
data_new=date_01_02.head(10)
data_new

,Date,Encode,Time,Price,Qty,State
0,TstUoB_2024-01-02,Exch0Lit,5.673,283,4,
1,TstUoB_2024-01-02,Exch0Lit,9.300,282,5,
2,TstUoB_2024-01-02,Exch0Lit,14.911,278,5,
3,TstUoB_2024-01-02,Exch0Lit,15.004,284,1,
4,TstUoB_2024-01-02,Exch0Lit,15.004,284,2,
5,TstUoB_2024-01-02,Exch0Lit,18.197,285,1,
6,TstUoB_2024-01-02,Exch0Lit,20.553,285,2,
7,TstUoB_2024-01-02,Exch0Lit,20.646,285,1,
8,TstUoB_2024-01-02,Exch0Lit,21.235,286,2,
9,TstUoB_2024-01-02,Exch0Lit,21.421,286,1,


In [45]:
# read the single file
with open('./Dataset/TstUoB_2024-01-02LOBs.txt','r') as f:
    lines=[]
    for line in f:
        lines.append(eval(line.strip('\n')))

In [46]:
# list to array
test_lines=np.array(lines)

/var/folders/vp/8_rd4_c56g3dpqb1167z6tqr0000gn/T/ipykernel_21462/1918146448.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_lines=np.array(lines)


***Reference:*** https://www.geeksforgeeks.org/python-convert-a-list-to-dictionary/

In [47]:
# Convert a list to dictionary
def Convert(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct

In [48]:
for i in test_lines:
    i[0]=[i[0],i[1]]
# df=pd.DataFrame(test_lines,columns=['time','bid','ask'],dtype=float)

In [49]:
test_lines=np.delete(test_lines, 1, 1) 

In [50]:
print(type(test_lines))

<class 'numpy.ndarray'>


In [51]:
test_lines[1:4]

array([[list(['time', 0.031]), list(['bid', []]), list(['ask', []])],
       [list(['time', 0.062]), list(['bid', []]), list(['ask', []])],
       [list(['time', 0.093]), list(['bid', []]), list(['ask', []])]],
      dtype=object)

In [52]:
for i in test_lines:
    i[0]=Convert(i[0])
    i[1]=Convert(i[1])
    i[2]=Convert(i[2])

***Reference:*** https://towardsdatascience.com/merge-dictionaries-in-python-d4e9ce137374#:~:text=Python%203.9%20has%20introduced%20the,a%20single%20line%20of%20code.&text=We%20can%20also%20merge%20the,update%20operator%20(%7C%3D).

In [22]:
for i in test_lines:
    i[0]=i[0]|i[1]|i[2]

print(test_lines[1])

[{'time': 0.031, 'bid': [], 'ask': []} {'bid': []} {'ask': []}]


In [23]:
test_lines=np.delete(test_lines,[1,2],1)

In [24]:
print(test_lines[1])

[{'time': 0.031, 'bid': [], 'ask': []}]


In [25]:
data_test=test_lines.flatten()

In [26]:
print(data_test[1])

{'time': 0.031, 'bid': [], 'ask': []}


In [27]:
data_test2=data_test.tolist()

In [28]:
print(data_test2[1])

{'time': 0.031, 'bid': [], 'ask': []}


In [50]:
df=pd.DataFrame(data_test2).rename(columns={"time":"Time"})
df3000=df.head(3000)

***Reference:*** https://pandas.pydata.org/docs/user_guide/merging.html

In [64]:
result = pd.merge(date_01_02,df, on='Time', how="inner")

In [65]:
def check_state(bid,ask):
    if bid==1 and ask==0:
        return 'bid'
    elif bid==0 and ask==1:
        return 'ask'
    elif bid==1 and ask==1:
        return 'bid/ask'
    else:
        return 'Nan'
    
    
for index, rows in result.iterrows():
    # if index<10:
    #     print(rows.bid)
    bid_in=0
    ask_in=0
    match_array=np.array([rows.Price,rows.Qty])
    bid_array=np.array(rows.bid)
    ask_array=np.array(rows.ask)
    for i in bid_array:
        if np.array_equal(match_array,i):
            bid_in=1
            break
        elif rows.Price==i[0]:
            index_df=df[df['Time']==rows.Time].index.values
            df_bid=df.loc[index_df[0]+1]["bid"]
            if np.any(np.isin(df_bid,rows.Price)):
                for k in df_bid:
                    if k[0]==i[0] and k[1]<i[1]:
                        bid_in=1
                        break
            else:
                bid_in=1
                break

    for j in ask_array:
        if np.array_equal(match_array,j):
            ask_in=1
            break
        elif rows.Price==j[0]:
            index_df=df[df['Time']==rows.Time].index.values
            df_ask=df.loc[index_df[0]+1]["ask"]
            if np.any(np.isin(df_ask,rows.Price)):
                for k in df_ask:
                    if k[0]==j[0] and k[1]<j[1]:
                        ask_in=1
                        break
            else:
                ask_in=1
                break
    result.at[index,'State']=check_state(bid_in,ask_in)



In [66]:
result.to_csv("./Processed_data/2024-01-02tapes.csv")